**Big Data**

**2020-2021**

**Programming Assignment # 1**

**Student Name: Tamer Tahamoqa**

**Student ID: 20208019**

**Guidelines:**

1. The assignment should be performed individually.
2. All submissions will be online (e-learning).
3. Assignment should include your full name and student ID.
4. Cutoff date for this homework is 05-1-2021 @ 12:00 PM.
5. Any late submission will be graded 0/100.
6. Academic Fraud: Cases of plagiarism will be handled according to university regulations.
7. Your homework should be solved as a Jupyter notebook file
8. You will submit a .ipynb file called YourName-ID.ipynb
9. Show all code in each step.



In [1]:
# Instantiating Spark Session
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()

---------------------

## Question 1 [50/100]

Download **Mosquitoes-Trap-Data** from this [URL](https://data.edmonton.ca/Environmental-Services/Mosquitoes-Trap-Data/5zeu-wkpv).

Write a complete PySpark application to answer the follwoing question. Make sure to define the schema and prvide all necessary spark transofrmation and action code to get to the answers.   
  
  - **Q1.1** Report all problems you can find in the dataset and how you fixed them.
  - **Q1.2** How many mosquitos caught in 2014?
  - **Q1.3** How many mosquitos of each species were caught?
  - **Q1.4** Which traps caught the most mosquitos?
  



### __Q1.1__ Report all problems you can find in the dataset and how you fixed them

In [2]:
# Inferring column names from the first row and inferring the data type on read
df_mosquito = spark.read.csv("Mosquitoes_Trap_Data__May_2011_to_Sep_2015_.csv", header=True, inferSchema=True)

In [3]:
df_mosquito.printSchema()

root
 |-- TRAP_DATE: string (nullable = true)
 |-- GENUS: string (nullable = true)
 |-- SPECIES: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- RURAL NORTH WEST: integer (nullable = true)
 |-- RURAL NORTH EAST: integer (nullable = true)
 |-- RURAL SOUTH EAST: integer (nullable = true)
 |-- RIVER VALLEY EAST: integer (nullable = true)
 |-- RIVER VALLEY WEST: integer (nullable = true)
 |-- RESIDENTIAL NORTH: integer (nullable = true)
 |-- RURAL SOUTH WEST: integer (nullable = true)
 |-- LAGOON: integer (nullable = true)
 |-- GOLF COURSE: integer (nullable = true)
 |-- INDUSTRIAL PARK: integer (nullable = true)
 |-- RESIDENTIAL SOUTH: integer (nullable = true)
 |-- TOTAL: integer (nullable = true)



In [4]:
df_mosquito.show(20)

+------------+-----+----------+-----------+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
|   TRAP_DATE|GENUS|   SPECIES|       TYPE|GENDER|RURAL NORTH WEST|RURAL NORTH EAST|RURAL SOUTH EAST|RIVER VALLEY EAST|RIVER VALLEY WEST|RESIDENTIAL NORTH|RURAL SOUTH WEST|LAGOON|GOLF COURSE|INDUSTRIAL PARK|RESIDENTIAL SOUTH|TOTAL|
+------------+-----+----------+-----------+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
| May 17 2011| null|      null|    Unknown|Female|            null|            null|            null|             null|             null|             null|            null|  null|       null|           null|             null| null|
| May 31 2011|Culex|      null|       null|Female|            null|     

In [5]:
df_mosquito.summary().show()

+-------+-----------------+--------+-----------+-----------+------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|        TRAP_DATE|   GENUS|    SPECIES|       TYPE|GENDER|  RURAL NORTH WEST| RURAL NORTH EAST| RURAL SOUTH EAST|RIVER VALLEY EAST| RIVER VALLEY WEST| RESIDENTIAL NORTH|  RURAL SOUTH WEST|            LAGOON|       GOLF COURSE|   INDUSTRIAL PARK| RESIDENTIAL SOUTH|             TOTAL|
+-------+-----------------+--------+-----------+-----------+------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             4386|    4182|       3672|       2958|  4386|               582|              517|              431

In [6]:
df_mosquito.select("SPECIES").distinct().count()

38

In [7]:
df_mosquito.select("SPECIES").distinct().show(38)

+------------+
|     SPECIES|
+------------+
|   intrudens|
|  increpitus|
|     punctor|
|      earlei|
|  hexodontus|
|    dorsalis|
|  minnesotae|
|     impiger|
|  mercurator|
|  cataphylla|
|    riparius|
|        null|
|   spencerii|
|   provocans|
|     pionips|
|  alaskensis|
|    tarsalis|
|    inornata|
| alaskaensis|
|    cinereus|
|  campestris|
|  flavescens|
|     fitchii|
|   impatiens|
|    incidens|
|  perturbans|
|    communis|
|nigromaculis|
|  excrucians|
|    pullatus|
|   territans|
|   morsitans|
|  canadensis|
|   sticticus|
|   diantaeus|
|  implicatus|
|      euedes|
|      vexans|
+------------+



In [8]:
# Counting number of missing (nan) values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df_mosquito.select([F.count(F.when(F.isnan(col), col)).alias(col) for col in df_mosquito.columns]).show()

+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
|TRAP_DATE|GENUS|SPECIES|TYPE|GENDER|RURAL NORTH WEST|RURAL NORTH EAST|RURAL SOUTH EAST|RIVER VALLEY EAST|RIVER VALLEY WEST|RESIDENTIAL NORTH|RURAL SOUTH WEST|LAGOON|GOLF COURSE|INDUSTRIAL PARK|RESIDENTIAL SOUTH|TOTAL|
+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
|        0|    0|      0|   0|     0|               0|               0|               0|                0|                0|                0|               0|     0|          0|              0|                0|    0|
+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+

In [9]:
# Counting number of null values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df_mosquito.select([F.count(F.when(F.isnull(col), col)).alias(col) for col in df_mosquito.columns]).show()

+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
|TRAP_DATE|GENUS|SPECIES|TYPE|GENDER|RURAL NORTH WEST|RURAL NORTH EAST|RURAL SOUTH EAST|RIVER VALLEY EAST|RIVER VALLEY WEST|RESIDENTIAL NORTH|RURAL SOUTH WEST|LAGOON|GOLF COURSE|INDUSTRIAL PARK|RESIDENTIAL SOUTH|TOTAL|
+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+
|        1|  205|    715|1429|     1|            3805|            3870|            3956|             4001|             4055|             4092|            3955|  4179|       4294|           4291|             4025|  267|
+---------+-----+-------+----+------+----------------+----------------+----------------+-----------------+-----------------+

In [10]:
# Checking number of duplicate rows
#  Used this stackoverflow answer: https://stackoverflow.com/a/48554666
df_mosquito.groupBy(df_mosquito.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

+----------+
|sum(count)|
+----------+
|      null|
+----------+



#### __Q1.1 Answer__:
* I have decided to let PySpark infer the schema and use the first row in the csv file as the column names.
* The dataset is plagued by a lot of null values due to trap failures.
* I have decided to not eliminate any record containing null values as that would practically eliminate almost the entire dataset.
* I tried to convert the 'TRAP_DATE' column to a PySpark DateType but I eventually decided to keep it in a string format.
* No duplicate rows have been found in the dataset

### __Q1.2__ How many mosquitoes caught in 2014?

In [11]:
df_mosquito = df_mosquito.withColumn("YEAR", F.split("TRAP_DATE", " ")[2])  # Add a 'Year' column

In [12]:
df_mosquito.show()

+------------+-----+----------+-----------+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+----+
|   TRAP_DATE|GENUS|   SPECIES|       TYPE|GENDER|RURAL NORTH WEST|RURAL NORTH EAST|RURAL SOUTH EAST|RIVER VALLEY EAST|RIVER VALLEY WEST|RESIDENTIAL NORTH|RURAL SOUTH WEST|LAGOON|GOLF COURSE|INDUSTRIAL PARK|RESIDENTIAL SOUTH|TOTAL|YEAR|
+------------+-----+----------+-----------+------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+------+-----------+---------------+-----------------+-----+----+
| May 17 2011| null|      null|    Unknown|Female|            null|            null|            null|             null|             null|             null|            null|  null|       null|           null|             null| null|2011|
| May 31 2011|Culex|      null|       null|Female|  

In [13]:
df_mosquito_2014 = df_mosquito.filter(df_mosquito["YEAR"] == "2014")

In [14]:
df_mosquito_2014.groupBy().sum().show()

+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|sum(RURAL NORTH WEST)|sum(RURAL NORTH EAST)|sum(RURAL SOUTH EAST)|sum(RIVER VALLEY EAST)|sum(RIVER VALLEY WEST)|sum(RESIDENTIAL NORTH)|sum(RURAL SOUTH WEST)|sum(LAGOON)|sum(GOLF COURSE)|sum(INDUSTRIAL PARK)|sum(RESIDENTIAL SOUTH)|sum(TOTAL)|
+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|                  667|                16753|                 4112|                  2588|                   959|                   647|                 3400|       null|            null|                2789|                  1758|     33673|
+---------------------+-----

In [15]:
df_mosquito_2014.groupBy().sum().select("sum(TOTAL)").show()

+----------+
|sum(TOTAL)|
+----------+
|     33673|
+----------+



#### __Q1.2 Answer: 33,673 mosquitoes__ were caught in 2014.

### __Q1.3__ How many mosquitoes of each species were caught?

In [16]:
df_mosquito.groupBy("Species").sum().show(38)  # 38 species (one for 'null' species value)

+------------+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|     Species|sum(RURAL NORTH WEST)|sum(RURAL NORTH EAST)|sum(RURAL SOUTH EAST)|sum(RIVER VALLEY EAST)|sum(RIVER VALLEY WEST)|sum(RESIDENTIAL NORTH)|sum(RURAL SOUTH WEST)|sum(LAGOON)|sum(GOLF COURSE)|sum(INDUSTRIAL PARK)|sum(RESIDENTIAL SOUTH)|sum(TOTAL)|
+------------+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|   intrudens|                    7|                 null|                 null|                  null|                  null|                  null|                 null|       null|            null|                null|           

#### __Q1.3 Answer__:

In [17]:
df_mosquito.groupBy("Species").sum().select("Species", "sum(TOTAL)").orderBy("sum(TOTAL)", ascending=False).show(38)  # 38 species (one for 'null' species value)

+------------+----------+
|     Species|sum(TOTAL)|
+------------+----------+
|      vexans|     75645|
|        null|     28010|
|    inornata|     11806|
|    dorsalis|      1124|
|  flavescens|       953|
|   spencerii|       923|
|      earlei|       523|
|   territans|       298|
|     fitchii|       283|
|  minnesotae|       197|
|  excrucians|       193|
|      euedes|       167|
|  perturbans|       157|
|  alaskensis|       141|
|    riparius|       104|
|   morsitans|       101|
|    incidens|        79|
|    tarsalis|        76|
|   impatiens|        55|
|  campestris|        45|
|nigromaculis|        12|
|   sticticus|        11|
|  increpitus|         8|
| alaskaensis|         8|
|   intrudens|         7|
|  implicatus|         4|
|  canadensis|         3|
|  mercurator|         2|
|    communis|         1|
|   provocans|         1|
|    cinereus|         1|
|     punctor|         0|
|  hexodontus|         0|
|     pionips|         0|
|     impiger|         0|
|  cataphyll

### __Q1.4__ Which traps caught the most mosquitoes?

Note: I am assuming this means which field had the most amount of caught mosquitoes.

In [18]:
df_mosquito.groupBy().sum().show()

+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|sum(RURAL NORTH WEST)|sum(RURAL NORTH EAST)|sum(RURAL SOUTH EAST)|sum(RIVER VALLEY EAST)|sum(RIVER VALLEY WEST)|sum(RESIDENTIAL NORTH)|sum(RURAL SOUTH WEST)|sum(LAGOON)|sum(GOLF COURSE)|sum(INDUSTRIAL PARK)|sum(RESIDENTIAL SOUTH)|sum(TOTAL)|
+---------------------+---------------------+---------------------+----------------------+----------------------+----------------------+---------------------+-----------+----------------+--------------------+----------------------+----------+
|                14555|                39507|                 7965|                  6987|                  5501|                  2712|                25660|       5678|            1788|                2789|                  7796|    120938|
+---------------------+-----

#### __Q1.4 Answer__: The __RURAL NORTH EAST__ location had the most amount of caught mosquitoes out of all locations.

-----------------

## Question 2 [50/100]

Download **LED Streetlight Conversion Data** from this [URL](https://data.edmonton.ca/Transportation/LED-Streetlight-Conversion/rxke-mcvd).

Write a complete PySpark application to answer the follwoing questions. Make sure to define the schema and provide all necessary spark transofrmations and actions  to get to the answers.   
  
  - **Q2.1** Report all problems you can find in the dataset and how you fixed them.
  - **Q2.2** How many total streetlights?
  - **Q2.3** How many streetlights are converted to LED?
  - **Q2.4** How many streetlights were converted by year?
  

### __Q2.1__ Report all problems you can find in the dataset and how you fixed them

In [19]:
# Inferring column names from the first row and inferring the data type on read
df_streetlight = spark.read.csv("LED_Streetlight_Conversion.csv", header=True, inferSchema=True)

In [20]:
df_streetlight.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Watts: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Ownership: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [21]:
df_streetlight.show(20)

+----+-----+----------------+---------+------------------+-------------------+--------------------+
|Type|Watts|      Start Date|Ownership|          Latitude|          Longitude|            Location|
+----+-----+----------------+---------+------------------+-------------------+--------------------+
| LED|  237|February 21 2020|      COE| 53.52467142817719|-113.61566462018001|(5932487.12484, 2...|
| LED|   22|December 19 2017|      COE| 53.47035024680297|-113.65637562043075|(5926428.2609995,...|
| LED|   43|December 19 2017|      COE|  53.4720334816896|-113.66173224778211|(5926613.87644131...|
| LED|   22|December 19 2017|      COE| 53.47083969350626|-113.65686969582687|(5926482.5703075,...|
| LED|   35|December 19 2017|      COE|53.470074318816785|-113.65843946794233|(5926396.89636846...|
| LED|   37|February 21 2020|      COE|53.523965853007276| -113.6148103041073|(5932408.9113597,...|
| LED|   43|December 19 2017|      COE| 53.47134418731741|-113.66215489764367|(5926537.03634357...|


In [22]:
df_streetlight.summary().show()

+-------+------+------------------+-----------------+---------+-------------------+-------------------+--------------------+
|summary|  Type|             Watts|       Start Date|Ownership|           Latitude|          Longitude|            Location|
+-------+------+------------------+-----------------+---------+-------------------+-------------------+--------------------+
|  count|117349|            117316|           117348|   117349|             117349|             117349|              117349|
|   mean|  null|105.95969006785093|             null|     null|  53.52645098249954|-113.51066671363635|                null|
| stddev|  null| 80.15090880880888|             null|     null|0.06082064301295191|0.07781926465097863|                null|
|    min|   HPS|                14|    April 01 1988|      COE|  53.33809790153047|-113.71311636184305|(5911740.64670252...|
|    25%|  null|                60|             null|     null|  53.47801453742578|-113.56654716458246|                null|


In [23]:
df_streetlight.select("Type").distinct().show()

+----+
|Type|
+----+
| INC|
| LED|
| HPS|
|  MV|
|  MH|
+----+



In [24]:
# Counting number of missing (nan) values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df_streetlight.select([F.count(F.when(F.isnan(col), col)).alias(col) for col in df_streetlight.columns]).show()

+----+-----+----------+---------+--------+---------+--------+
|Type|Watts|Start Date|Ownership|Latitude|Longitude|Location|
+----+-----+----------+---------+--------+---------+--------+
|   0|    0|         0|        0|       0|        0|       0|
+----+-----+----------+---------+--------+---------+--------+



In [25]:
# Counting number of null values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df_streetlight.select([F.count(F.when(F.isnull(col), col)).alias(col) for col in df_streetlight.columns]).show()

+----+-----+----------+---------+--------+---------+--------+
|Type|Watts|Start Date|Ownership|Latitude|Longitude|Location|
+----+-----+----------+---------+--------+---------+--------+
|   0|   33|         1|        0|       0|        0|       0|
+----+-----+----------+---------+--------+---------+--------+



In [26]:
# Checking number of duplicate rows
#  Used this stackoverflow answer: https://stackoverflow.com/a/48554666
df_streetlight.groupBy(df_streetlight.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

+----------+
|sum(count)|
+----------+
|      4749|
+----------+



In [27]:
# Removing duplicate rows
df_streetlight = df_streetlight.dropDuplicates()

In [28]:
df_streetlight.count()

114902

#### __Q2.1 Answer__:
* I have decided to let PySpark infer the schema and use the first row in the csv file as the column names.
* There is one record with a null date value and 33 records with null watts values.
* I have decided to not eliminate any record containing null values for the analysis.
* 4749 duplicate rows have been found and removed from the dataset

### __Q2.2__ How many total streetlights?

In [29]:
df_streetlight.count()

114902

#### __Q2.2 Answer__: 114,902 streetlights

### __Q2.3__ How many streetlights are converted to LED?

In [30]:
df_streetlight.filter(df_streetlight["Type"] == "LED").count()

50492

#### __Q2.3 Answer__: 50,492 streetlights were converted to LED

### __Q2.4__ How many streetlights were converted by year?

In [31]:
df_streetlight = df_streetlight.withColumn("Year", F.split("Start Date", " ")[2])  # Add a 'Year' column

In [32]:
df_streetlight.show()

+----+-----+-----------------+---------+------------------+-------------------+--------------------+----+
|Type|Watts|       Start Date|Ownership|          Latitude|          Longitude|            Location|Year|
+----+-----+-----------------+---------+------------------+-------------------+--------------------+----+
| LED|   51|  January 15 2018|      COE| 53.53287863556411|-113.46888339036123|(5933462.98184977...|2018|
| LED|   51|  January 15 2018|      COE| 53.53232697346642|-113.47003087340482|(5933401.0242782,...|2018|
| LED|   72|     July 24 2013|      COE| 53.52836144525659|-113.64023322880963|(5932889.25894843...|2013|
| LED|   43| December 23 2011|      COE|53.544666080163026|-113.45023087385474|(5934784.11410638...|2011|
| LED|   34|  January 01 2015|      COE| 53.59385871760341|-113.52498558573531|(5940222.86090743...|2015|
| LED|   34|  January 01 2015|      COE|  53.5982783677076|-113.53071237680531|(5940712.18804645...|2015|
| LED|   65|  January 25 2016|      COE| 53.63

In [33]:
df_streetlight.select("Year").distinct().count()

49

In [34]:
df_streetlight.select("Year").distinct().orderBy("Year").show(49)

+----+
|Year|
+----+
|null|
|1974|
|1976|
|1977|
|1978|
|1979|
|1980|
|1981|
|1982|
|1983|
|1984|
|1985|
|1986|
|1987|
|1988|
|1989|
|1990|
|1991|
|1992|
|1993|
|1994|
|1995|
|1996|
|1997|
|1998|
|1999|
|2000|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2007|
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2027|
|3008|
+----+



#### __Q2.4 Answer__:

In [35]:
df_streetlight.groupBy("Year").count().orderBy("count", ascending=False).show(49)  # 49 distinct years (one for 'null' year value)

+----+-----+
|Year|count|
+----+-----+
|2002|14062|
|2016|12470|
|2012| 8088|
|2018| 7136|
|2015| 6411|
|2011| 5927|
|2013| 5600|
|2017| 4631|
|2014| 4549|
|2019| 3114|
|2008| 2866|
|2010| 2706|
|2003| 2524|
|1985| 2402|
|2005| 2352|
|2004| 2087|
|2007| 2055|
|1990| 1764|
|1991| 1718|
|1993| 1688|
|2009| 1623|
|1989| 1594|
|2006| 1587|
|1992| 1376|
|1984| 1259|
|2000| 1244|
|1994| 1242|
|1998| 1168|
|2001| 1160|
|1999| 1064|
|1986| 1051|
|1988|  984|
|2020|  869|
|1997|  746|
|1995|  744|
|1980|  619|
|1987|  589|
|1979|  424|
|1982|  399|
|1996|  390|
|1981|  216|
|1983|  195|
|1978|   96|
|1977|   85|
|1976|   24|
|null|    1|
|1974|    1|
|3008|    1|
|2027|    1|
+----+-----+

